In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
import re
import time

In [2]:
# 파일 불러오기: 불러올 파일을 현재 폴더에 넣어두세요.
data = pd.read_csv('programmers.csv')  # 파일명만 변경하여 그대로 돌리시면 됩니다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   기업명          48 non-null     object
 1   사원수          48 non-null     object
 2   주소           48 non-null     object
 3   직무           48 non-null     object
 4   이용하는 기술스택    48 non-null     object
 5   자격요건         47 non-null     object
 6   직급           48 non-null     object
 7   우대사항         47 non-null     object
 8   해당 페이지 URL   48 non-null     object
 9   기업 홈페이지 URL  48 non-null     object
dtypes: object(10)
memory usage: 3.9+ KB


In [3]:
# 기업개황 검색하기 위하여 정식 회사명 만들기 + 공백 제거
corp_name_list = data.기업명

last_res = pd.DataFrame(
    columns = ['기업명', '업종분류', '사원수', '설립연도', '매출액']
    )  # '기업 홈페이지 URL' 값이 출력안돼서 제거

# 크롬옵션
chrome_options = Options()
# 꺼짐 방지 (gpu 가속 금지)
chrome_options.add_argument('--disable-gpu') 
# 전체 창 고정
chrome_options.add_argument("--start-maximized")
# 알림표시 제거
chrome_options.add_argument("--disable-notifications")
# 옵션 적용
driver = webdriver.Chrome(options=chrome_options)
wan_ins_url = '''https://insight.wanted.co.kr/'''
driver.get(wan_ins_url)





for corp_name in corp_name_list:
    time.sleep(0.5)
    # 기업 검색창 클릭
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="__next"]/div[1]/nav/div/div[2]/a/div[2]'))).click()

    # 기업 입력
    driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/form/div/div/input').send_keys(f'{corp_name}')
    time.sleep(1.2)
    
    #엔터 입력
    driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/form/div/div/input').send_keys(Keys.ENTER)
    
    # 기업입력 결과를 5초간 기다림
    try:
        WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="summary"]')))
    # 클릭 불가능시 처리 다음 반복 진행 코드
    except ElementClickInterceptedException:
        driver.get(wan_ins_url)
        continue
    # 처리시간 길어질 시 다음 반복 진행 코드
    except TimeoutException:
        driver.get(wan_ins_url)
        continue

    # 기본값을 None으로 하고 값이 있으면 가져오고 없으면 pass하는 방식으로 처리
    # url은 값이 없으면 - 이 출력되기 때문에 일괄처리를 위하여 -를 출력하는 방식으로 처리
    employee = None
    sales_money = None
    sector = None
    corp_year = None
    corp_url = '-'
    
    # Crawling-data
    # 사원 항목이 존재하냐 안하냐에 따라 XPATH가 다른것을 확인하여 try, except로 처리
    try:    
        employee = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="summary"]/div[2]/div[1]/div[2]/div[2]/div[1]/span'))).text
    except:
        pass    
    try:    
        sales_money = re.sub('\s', '', driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[4]/dd').text)
    except:
        sales_money = re.sub('\s', '', driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div/dl[4]/dd').text)
    try:
        sector = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[1]/dd').text
    except:
        sector = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div/dl[1]/dd').text
 
    try:
        corp_year = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div/dl[2]/dd').text
    except:
        corp_year = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[2]/dd').text
    try:    
        corp_url = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div[2]/dl[8]/dd').text
    except:
        corp_url = driver.find_element(By.XPATH, '//*[@id="summary"]/div[2]/div/dl[8]/dd').text
    
    # Data concat
    last_temp = pd.DataFrame(
        [corp_name, sector, employee, corp_year, sales_money],
            index = ['기업명', '업종분류', '사원수', '설립연도', '매출액']
        ).T
    last_res = pd.concat([last_res, last_temp])
    driver.back()
    time.sleep(0.2)
    
    driver.quit()

In [5]:
last_res.to_csv('./wantedinsight_result.csv')